In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util


In [ ]:
df = pd.read_csv("college_3.0.csv")
df = df.dropna(subset=['Name', 'Location', 'Majors_Offered'])
df["full_text"] = df.apply(
    lambda row: f"Name: {row['Name']}. Location: {row['Location']}. Majors: {row['Majors_Offered']}. Hostel: {row.get('Hostel', 'Not specified')}.",
    axis=1
)


In [ ]:
# embedding
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = embedder.encode(df["full_text"].tolist(), convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load model
from huggingface_hub import login
login()

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def answer_query(query):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(similarity_scores, k=5)
    top_texts = [df["full_text"][i] for i in top_results[1]]
    context = "\n".join(top_texts)
    prompt = f"Answer the following question using only the information provided below. Give a detailed, clear, and friendly answer within 100 words.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    response = generator(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)[0]['generated_text']
    answer_start = response.find("Answer:") + len("Answer:")
    return response[answer_start:].strip()


In [ ]:
print("Welcome to the College Info Chatbot...type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break
    try:
        response = answer_query(user_input)
        print(f"Bot: {response}\n")
    except Exception as e:
        print(f"Bot: Sorry, an error occurred: {e}\n")


Welcome to the College Info Chatbot! Type 'exit' to quit.

You: Which colleges in Chennai offer AI and have hostel?
Bot: Sorry, an error occurred: iteration over a 0-d tensor



KeyboardInterrupt: Interrupted by user